Solve using Python with the customer service data json file:

1. **Data Transformation and Aggregation:**

   Write Python code to transform the customer service data into a new JSON structure where each customer is represented as a dictionary with keys like "customer_id," "customer_name," "total_tickets_open," and "last_ticket_status." Calculate the total number of open tickets for each customer and include the status of their most recent ticket.

2. **Data Filtering and Validation:**

   Create a Python function that filters the customer service data to find customers who have open support tickets (status is "Open"). Ensure that the function handles missing or invalid data gracefully and provides appropriate error messages.

3. **Customer with Most Tickets:**

   Write Python code to identify and print the customer with the most support tickets. Provide their name, email address, and the number of tickets they have opened.

4. **Ticket Resolution Time:**

   Calculate the average resolution time for closed support tickets in hours. This involves calculating the time difference between the "date_opened" and "date_closed" for each closed ticket, and then finding the average.

5. **Subscription-Based Analysis:**

   Create a Python function that analyzes the data to find the most common SaaS product among customers who have open support tickets. Provide the product name and the count of customers using that product with open tickets.

These questions assess a candidate's ability to manipulate and analyze data using Python, handle data validation and filtering, and perform calculations based on date and time data. Candidates may be asked to provide both the code and a brief explanation of their approach during the interview process.

In [15]:
import pandas as pd
import json
# Reading data from JSON file
with open("customer_service_data.json", "r") as file:
    customer_service_data = json.load(file)

Now, let's find number of open tickets for each customer and the last status

In [46]:
output = []
for customer_info in customer_service_data["customer_service"]:
    customer = {
        "customer_id": customer_info["customer_id"],
        "customer_name": customer_info["customer_name"],
        "total_tickets_open": 0,
        "last_ticket_status": None
    }
    if customer_info["support_tickets"]:
        open_tickets = [ticket for ticket in customer_info["support_tickets"] if ticket["status"] == "Open"]
        customer["total_tickets_open"] = len(open_tickets)
        if open_tickets:
            customer["last_ticket_status"] = open_tickets[-1]["status"]
    output.append(customer)
print(output)

[{'customer_id': '1001', 'customer_name': 'John Doe', 'total_tickets_open': 2, 'last_ticket_status': 'Open'}, {'customer_id': '1002', 'customer_name': 'Jane Smith', 'total_tickets_open': 1, 'last_ticket_status': 'Open'}, {'customer_id': '1003', 'customer_name': 'Bob Johnson', 'total_tickets_open': 0, 'last_ticket_status': None}]


Next, we will write a python function which finds all customers with open tickets 

In [47]:
cs_data = pd.DataFrame(customer_service_data)
def open_ticket_customers(customer_service_data: pd.DataFrame):
    result = []
    for customer_info in customer_service_data["customer_service"]:
        customer = {
            "customer_id": customer_info["customer_id"],
            "customer_name": customer_info["customer_name"],
            "total_tickets_open": 0,
        }
        if customer_info["support_tickets"]:
            open_tickets = [ticket for ticket in customer_info["support_tickets"] if ticket["status"] == "Open"]
            customer["total_tickets_open"] = len(open_tickets)
        result.append(customer)
    return result
output = pd.DataFrame(open_ticket_customers(cs_data))
output = output[output["total_tickets_open"] >0]
original_data = pd.DataFrame([c for c in cs_data["customer_service"]])
final = pd.merge(original_data,output, how="inner", on="customer_id")
final

,customer_id,customer_name_x,customer_email,subscription_id,product_name,support_tickets,customer_name_y,total_tickets_open
0,1001,John Doe,johndoe@example.com,S12345,SaaS Product X,"[{'ticket_id': 'T001', 'date_opened': '2023-09...",John Doe,2
1,1002,Jane Smith,janesmith@example.com,S12346,SaaS Product Y,"[{'ticket_id': 'T003', 'date_opened': '2023-09...",Jane Smith,1


Let's find out the average ticket resolution time and the fastest ticket that was resolved

In [5]:
from datetime import datetime
ticket_resolution =[]
for customer_info in customer_service_data["customer_service"]:
    for ticket in customer_info["support_tickets"]:
        if customer_info["support_tickets"] and ticket["status"] == "Closed":
            duration = (datetime.strptime(ticket["date_closed"], "%Y-%m-%d %H:%M:%S") - datetime.strptime(ticket["date_opened"], "%Y-%m-%d %H:%M:%S"))
            ticket_resolution.append(duration.seconds // 60 % 60)

print(ticket_resolution)
average_duration = sum(ticket_resolution) / len(ticket_resolution)
print("Average Ticket resolution (minutes):" ,average_duration)

[15, 30, 30]
Average Ticket resolution (minutes): 25.0


Find products with open tickets and count of customers

In [20]:
from collections import defaultdict
product_info = defaultdict(lambda: {"open_ticket_count": 0, "customer_count": 0})
for customer_info in customer_service_data["customer_service"]:
    open_tickets = [ticket for ticket in customer_info["support_tickets"] if ticket["status"] == "Open"]
    if open_tickets:
        product_name = customer_info["product_name"]
        product_info[product_name]["open_ticket_count"] += len(open_tickets)
        product_info[product_name]["customer_count"] += 1

product_info

defaultdict(<function __main__.<lambda>()>,
            {'SaaS Product X': {'open_ticket_count': 4, 'customer_count': 2},
             'SaaS Product Y': {'open_ticket_count': 2, 'customer_count': 2}})